I use rpy2 for some operations be completed using R syntax.

In [ ]:
# sudo pip3 install nltk
# sudo pip3 install pandas
# sudo pip3 install rpy2 && sudo pip3 install gensim
# sudo pip3 install matplotlib


In [1]:

import nltk
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer


import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
import rpy2.robjects.numpy2ri
from rpy2.robjects import pandas2ri
pandas2ri.activate()

# tidyr = importr('tidyverse')
# magrittr = importr('magrittr')

/usr/local/lib/python3.5/dist-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: During startup - 
  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.5/dist-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Warning messages:

  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.5/dist-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: 1: Setting LC_TIME failed, using "C" 

  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.5/dist-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: 2: Setting LC_MONETARY failed, using "C" 

  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.5/dist-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: 3: Setting LC_PAPER failed, using "C" 

  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.5/dist-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: 4: Setting LC_MEASUREMENT failed, using "C" 

  warnings.warn(x, RRuntimeWarning)


In [ ]:
Building a single courpus

In [12]:
# rstring = """
#     function(){  
    
#     l = list.files("~/imdb/data/train/pos/", full.names = T)

#     t = data.frame()

#     for (i in 1:length(l)){
#       t = rbind(t, data.frame(path = l[i], 
#                      text = readLines(l[i])))
#     }
    
#     t
    
#     }
# """

# rfunc = robjects.r(rstring)
# r_df = rfunc()
# df = pandas2ri.ri2py(r_df)


# df.to_csv("~/imdb/data/df_texts.csv")

In [2]:
df = pd.read_csv("~/imdb/data/df_texts.csv")

### Corpus

In [3]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [1]:
# Transform to lower case
import re
corpus = df["text"].str.lower()
corpus = re.sub('[0-9]+', '', corpus)


NameError: name 'df' is not defined

## Tokenisation

In [5]:
## Removing punctuation

tokenizer = RegexpTokenizer(r'\w+')
sentences = [tokenizer.tokenize(doc) for doc in corpus]

# Stemming and Lemmatisation

## stemming
porter_stemmer = PorterStemmer()

## lemmatisation
wordnet_lemmatizer = WordNetLemmatizer()


sentences = [[wordnet_lemmatizer.lemmatize(porter_stemmer.stem(token)) for token in sentence] for sentence in sentences]


### ML part

Change log format

In [6]:
import logging
logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)
logging.root.level = logging.INFO 

In [7]:
import gensim
from gensim.models import Word2Vec

INFO : 'pattern' package not found; tag filters are not available for English


parameters:

- sentences - data
- sg - algorirthms: 0 - CBOW, 1 - Skip-gram
- size - vector size, dimensionality 
- window - windows size
- min_count - threshold of word count to be in a vocabulary
- max_vocab_size - keeping more-less frequent words
- sample - downsampling of frequent words. Smaller value - less probability to keep it
- alpha - learning rate
- min_alpha - minimum learning rate
- negative - negative examples # higher values are appropriate for a smaller corpus
- iter -- number of iterations

Practical difference between CBOW and Skip-gram is that the latter is better dealing with less frequent words.

In [10]:
model = Word2Vec(sentences,
                    sg = 1,
                    window = 10,
                    min_count = 20,
                    seed = 6,
                    workers = 4,
                    size = 150)

INFO : collecting all words and their counts
INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO : PROGRESS: at sentence #10000, processed 2442598 words, keeping 33760 word types
INFO : collected 37617 word types from a corpus of 3060552 raw words and 12500 sentences
INFO : Loading a fresh vocabulary
INFO : min_count=20 retains 7048 unique words (18% of original 37617, drops 30569)
INFO : min_count=20 leaves 2945923 word corpus (96% of original 3060552, drops 114629)
INFO : deleting the raw counts dictionary of 37617 items
INFO : sample=0.001 downsamples 49 most-common words
INFO : downsampling leaves estimated 2137631 word corpus (72.6% of prior 2945923)
INFO : estimated required memory for 7048 words and 150 dimensions: 11981600 bytes
INFO : resetting layer weights
INFO : training model with 4 workers on 7048 vocabulary and 150 features, using sg=1 hs=0 sample=0.001 negative=5 window=10
INFO : PROGRESS: at 1.09% examples, 110354 words/s, in_qsize 7, out_qsiz

Saving and loading

In [12]:
model.save('~/imdb/models/model.bin')
model.wv.save_word2vec_format('~/imdb/models/model.wv')

# save_word2vec_format
# gensim.models.Word2Vec.load('~/imdb/models/model.bin')

INFO : saving Word2Vec object under ~/imdb/model.bin, separately None
INFO : not storing attribute syn0norm
INFO : not storing attribute cum_table
INFO : saved ~/imdb/model.bin
INFO : storing 7048x150 projection weights into ~/imdb/model.wv
INFO : loading projection weights from ~/imdb/model.wv
INFO : loaded (7048, 150) matrix from ~/imdb/model.wv


In [7]:
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format('~/imdb/models/model.wv')

INFO : loading projection weights from ~/imdb/models/model.wv
INFO : loaded (7048, 150) matrix from ~/imdb/models/model.wv


## Semantic Arithmetic

In [9]:
model.most_similar('actor',  topn = 15)

[('actress', 0.6813883781433105),
 ('cast', 0.6340926289558411),
 ('stoltz', 0.5940563678741455),
 ('calib', 0.5839543342590332),
 ('ron', 0.583796501159668),
 ('versatil', 0.5788587927818298),
 ('phenomen', 0.5738711357116699),
 ('relish', 0.5677598118782043),
 ('ensembl', 0.5595935583114624),
 ('smallest', 0.5581124424934387),
 ('recogniz', 0.557616651058197),
 ('orlando', 0.5566136240959167),
 ('deniro', 0.5559999942779541),
 ('comedian', 0.5505030155181885),
 ('forb', 0.5500813722610474)]

In [10]:
model.similarity('actor', "actress")

0.6813883503689548

In [12]:
model.doesnt_match('actor actrees scene'.split())

'scene'

In [19]:
## As it was seen before, corpus is overloaded with names - semantic algebra here is not working like we might expect

print(model.most_similar(positive = ['actor', 'actress'], negative = ['man'])[0][0])

stoltz


## Refine a model - CBOW

In [8]:
model = Word2Vec(sentences,
                    sg = 0,
                    window = 10,
                    min_count = 20,
                    seed = 6,
                    workers = 4,
                    size = 150)
model.save('~/imdb/models/model_cbow.bin')

INFO : collecting all words and their counts
INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO : PROGRESS: at sentence #10000, processed 2442598 words, keeping 33760 word types
INFO : collected 37617 word types from a corpus of 3060552 raw words and 12500 sentences
INFO : Loading a fresh vocabulary
INFO : min_count=20 retains 7048 unique words (18% of original 37617, drops 30569)
INFO : min_count=20 leaves 2945923 word corpus (96% of original 3060552, drops 114629)
INFO : deleting the raw counts dictionary of 37617 items
INFO : sample=0.001 downsamples 49 most-common words
INFO : downsampling leaves estimated 2137631 word corpus (72.6% of prior 2945923)
INFO : estimated required memory for 7048 words and 150 dimensions: 11981600 bytes
INFO : resetting layer weights
INFO : training model with 4 workers on 7048 vocabulary and 150 features, using sg=0 hs=0 sample=0.001 negative=5 window=10
INFO : PROGRESS: at 3.41% examples, 342113 words/s, in_qsize 8, out_qsiz

In [12]:
from gensim.models import KeyedVectors
model = gensim.models.Word2Vec.load('~/imdb/models/model_cbow.bin')
print(model.most_similar(positive = ['actor', 'actress'], negative = ['man'])[0][0])

INFO : loading Word2Vec object from ~/imdb/models/model_cbow.bin
INFO : loading wv recursively from ~/imdb/models/model_cbow.bin.wv.* with mmap=None
INFO : setting ignored attribute syn0norm to None
INFO : setting ignored attribute cum_table to None
INFO : loaded ~/imdb/models/model_cbow.bin
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  This is separate from the ipykernel package so we can avoid doing imports until
INFO : precomputing L2-norms of word weight vectors


cast
